In [ ]:
#conda install -c conda-forge \
  #num#py pandas xarray geopandas rasterio matplotlib rioxarray tqdm \
  #scikit-learn joblib statsmodels scipy imageio optuna pmdarima

In [ ]:
#python -c "import numpy, pandas, xarray, geopandas, rasterio, matplotlib, rioxarray, tqdm, sklearn, joblib, statsmodels, scipy, imageio, optuna, pmdarima, tensorflow; print('All OK!')"

In [ ]:
"""
Landslide Prediction Spatio-Temporal Pipeline (v5, improved)
------------------------------------------------------------
Pipeline otomatis prediksi spasio-temporal kejadian longsor grid dengan ARIMA, LSTM, RF, ANN:
- Input: NetCDF grid, shapefile titik longsor (opsional)
- Feature engineering: lag, spasial rata-rata, sin/cos waktu, dst
- Hyperparameter tuning otomatis (Optuna)
- Output: GeoTIFF, NetCDF, PNG, GIF, NPZ, log, residual diagnostics, evaluasi grid lengkap
- Logging detail & validasi data
- File output otomatis pakai hash param model
- Support paralelisme CPU, robust handling data NaN/missing
- Penambahan: ANN aktif, residual train grid, colormap configurable, file output pakai hash param, units/desc custom, validasi NaN, improved exception handling, docstring global.
"""

import os
import random
import hashlib
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import rioxarray
import warnings
import logging
from rasterio import features
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from joblib import Parallel, delayed
from statsmodels.tsa.stattools import adfuller, acf
import pmdarima as pm
from scipy.ndimage import convolve
import optuna
import imageio
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Ignore warnings for clean output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
plt.ioff()

# ========================= CONFIG =========================
CONFIG = {
    "BASE_DIR": "D:\\DataPenelitian\\Longsor",
    "NC_FILE": "nc_20250426_1M.nc",
    "LANDSLIDE_SHP_FILE": "data_fix/TitikLongsor_Magelang_2025.shp",
    "LANDSLIDE_DATE_COLUMN": "Date",
    "TARGET_VARIABLE": "COUNT",
    "LANDSLIDE_FEATURE_NAME": "LANDSLIDE_COUNT",
    "SPATIAL_FEATURE_NAME": "COUNT_SPATIAL_AVG_LAG1",
    "TARGET_CRS": "EPSG:32749",
    "SAMPLE_COORDS": [(0, 1), (35, 14), (0, 0), (27, 6), (35, 11)],
    "N_LAG": None,
    "FORECAST_STEPS": None,
    "VALIDATION_GAP": None,
    "HYPERPARAM_TUNE": True,
    "OPTUNA_TRIALS": 15,
    "N_JOBS_PARALLEL": -1,
    "GIF_FPS": 2,
    "SEED": 42,
    "COLORMAP": "viridis",
    "COLORMAP_R2": "viridis_r"
}
MODEL_LIST = ["ann"] #, "lstm", "rf", "ann"]  # ANN sekarang aktif
OUTPUT_FOLDERS = {
    "geotiff": "output_geotiff",
    "netcdf": "output_netcdf",
    "png": "output_png",
    "gif": "output_gif",
    "logs": "logs",
    "intermediate": "output_intermediate"
}

In [ ]:

# ========== Setup Seed ==========
os.environ["PYTHONHASHSEED"] = str(CONFIG["SEED"])
random.seed(CONFIG["SEED"])
np.random.seed(CONFIG["SEED"])
tf.random.set_seed(CONFIG["SEED"])

def hash_params(params: dict) -> str:
    if not params: return "default"
    s = str(sorted(params.items()))
    return hashlib.md5(s.encode()).hexdigest()[:8]

def setup_folders(base_dir, folders):
    out_dirs = {}
    for k, v in folders.items():
        d = os.path.join(base_dir, v)
        os.makedirs(d, exist_ok=True)
        out_dirs[k] = d
    return out_dirs
OUTDIRS = setup_folders(CONFIG["BASE_DIR"], OUTPUT_FOLDERS)

def setup_logging(log_dir):
    log_path = os.path.join(log_dir, "run.log")
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s:%(message)s',
        handlers=[
            logging.FileHandler(log_path, mode='w'),
            logging.StreamHandler()
        ]
    )
    logging.info("Logging started. All logs saved to %s", log_path)
setup_logging(OUTDIRS["logs"])

def robust_impute(ts):
    s = pd.Series(ts)
    s = s.interpolate(method='linear', limit_direction='both').fillna(method='bfill').fillna(method='ffill').fillna(0)
    return s.values

def fallback_crs(ds, config):
    try:
        if not hasattr(ds, 'rio') or ds.rio.crs is None:
            ds = ds.rio.write_crs(config["TARGET_CRS"])
            logging.info(f"Fallback CRS applied: {config['TARGET_CRS']}")
    except Exception as e:
        logging.warning(f"Fallback CRS failed: {e}")
    return ds

def calculate_spatial_avg_lag1(data_array, feature_name="SPATIAL_AVG_LAG1"):
    kernel = np.array([[1,1,1],[1,0,1],[1,1,1]], dtype=np.float32)
    values = data_array.values.astype(np.float32)
    spatial_avg_lag1 = np.full_like(values, np.nan)
    for t in range(1, values.shape[0]):
        data_t_minus_1 = values[t-1,:,:]
        nan_mask_t_minus_1 = np.isnan(data_t_minus_1)
        neighbor_counts = convolve(~nan_mask_t_minus_1, kernel, mode='constant', cval=0.0)
        data_t_minus_1_nan_as_zero = np.nan_to_num(data_t_minus_1, nan=0.0)
        neighbor_sum = convolve(data_t_minus_1_nan_as_zero, kernel, mode='constant', cval=0.0)
        valid_neighbors_mask = neighbor_counts > 0
        avg_values = np.full_like(neighbor_sum, np.nan)
        avg_values[valid_neighbors_mask] = neighbor_sum[valid_neighbors_mask] / neighbor_counts[valid_neighbors_mask]
        spatial_avg_lag1[t,:,:] = avg_values
    spatial_da = xr.DataArray(
        spatial_avg_lag1, coords=data_array.coords, dims=data_array.dims, name=feature_name,
        attrs={'long_name': f'Average of 8 spatial neighbors at lag 1 for {data_array.name}',
               'units': data_array.attrs.get('units', 'unknown')}
    )
    logging.info(f"Calculated spatial feature: {feature_name}")
    return spatial_da

def add_time_features(ds, time_var='time'):
    try:
        months = pd.to_datetime(ds[time_var].values).month
        ds['month'] = (ds[time_var].dims, months)
        ds['sin_month'] = (ds[time_var].dims, np.sin(2 * np.pi * months / 12))
        ds['cos_month'] = (ds[time_var].dims, np.cos(2 * np.pi * months / 12))
        logging.info("Added time features: month, sin_month, cos_month.")
    except Exception as e:
        logging.warning(f"Could not add time features: {e}")
    return ds

def detect_agg_label(ds):
    time = pd.to_datetime(ds['time'].values)
    if len(time) < 2:
        return "1m"
    delta = (time[1] - time[0]).days
    if 25 <= delta < 35:
        return "1m"
    elif 55 <= delta < 65:
        return "2m"
    elif 85 <= delta < 95:
        return "3m"
    else:
        return f"{delta}d"

def forecast_filename(model, agg_label, start_date, end_date, step_ext):
    return f"{model}_forecast_{agg_label}_{start_date}_to_{end_date}_{step_ext}"

def evalmap_filename(model, agg_label, metric, ext):
    return f"{model}_eval_{metric}_{agg_label}.{ext}"

def difforder_filename(model, agg_label, ext):
    return f"{model}_orderdiff_{agg_label}.{ext}"

def residualdiag_filename(model, agg_label, ext):
    return f"{model}_residual_diag_{agg_label}.{ext}"

def process_landslide_data(shp_path, date_column, ds_template, feature_name="LANDSLIDE_COUNT"):
    try:
        landslide_gdf = gpd.read_file(shp_path)
        logging.info(f"Loaded {len(landslide_gdf)} landslide points from {os.path.basename(shp_path)}.")
        if date_column not in landslide_gdf.columns:
            logging.error(f"Date column '{date_column}' not found in Shapefile."); return None
        landslide_gdf[date_column] = pd.to_datetime(landslide_gdf[date_column], errors='coerce')
        original_count = len(landslide_gdf)
        landslide_gdf = landslide_gdf.dropna(subset=[date_column])
        if len(landslide_gdf) < original_count:
            logging.warning(f"Removed {original_count - len(landslide_gdf)} points with invalid dates.")
        if landslide_gdf.empty:
            logging.warning("No valid landslide points. Returning zero array.")
            return xr.DataArray(
                np.zeros((len(ds_template['time']), ds_template.dims['y'], ds_template.dims['x']), dtype=np.int32),
                coords=ds_template.coords, dims=['time', 'y', 'x'], name=feature_name
            )
        ds_template_crs = ds_template.rio.crs if hasattr(ds_template, 'rio') and ds_template.rio.crs else None
        if ds_template_crs and landslide_gdf.crs and landslide_gdf.crs != ds_template_crs:
            logging.info(f"Reprojecting landslide data from {landslide_gdf.crs} to {ds_template_crs}...")
            landslide_gdf = landslide_gdf.to_crs(ds_template_crs)
        ny, nx = ds_template.dims['y'], ds_template.dims['x']
        grid_shape = (ny, nx)
        transform = ds_template.rio.transform()
        time_coords = ds_template['time']
        landslide_grid_monthly = np.zeros((len(time_coords), ny, nx), dtype=np.int32)
        landslide_gdf['YearMonth'] = landslide_gdf[date_column].dt.to_period('M')
        for t_idx, timestamp in enumerate(tqdm(time_coords.values, desc="Rasterizing Landslides")):
            current_month = pd.Timestamp(timestamp).to_period('M')
            monthly_points = landslide_gdf[landslide_gdf['YearMonth'] == current_month]
            if not monthly_points.empty:
                shapes = [(geom, 1) for geom in monthly_points.geometry]
                try:
                    monthly_raster = features.rasterize(
                        shapes=shapes, out_shape=grid_shape, transform=transform,
                        fill=0, merge_alg=rasterio.enums.MergeAlg.add, dtype=np.int32)
                    landslide_grid_monthly[t_idx, :, :] = monthly_raster
                except Exception as raster_err:
                    logging.warning(f"Rasterize failed for month {current_month}: {raster_err}")
        landslide_da = xr.DataArray(
            landslide_grid_monthly, coords={'time': time_coords, 'y': ds_template['y'], 'x': ds_template['x']},
            dims=['time', 'y', 'x'], name=feature_name,
            attrs={'long_name': 'Monthly landslide event count', 'units': 'count'}
        )
        logging.info(f"Landslide data processing complete. Feature: {feature_name}")
        return landslide_da
    except Exception as e:
        logging.error(f"Error in process_landslide_data: {e}")
        return None

def set_window_params(ds, config):
    n_time = ds[config["TARGET_VARIABLE"]].shape[0]
    forecast_steps = config.get("FORECAST_STEPS")
    if forecast_steps is None:
        forecast_steps = max(1, int(0.1 * n_time))
    forecast_steps = min(forecast_steps, max(1, int(0.5 * n_time)))
    n_lag = config.get("N_LAG")
    if n_lag is None:
        n_lag = max(1, int(0.25 * n_time))
    n_lag = min(n_lag, max(1, int(1/3 * n_time)))
    validation_gap = config.get("VALIDATION_GAP")
    if validation_gap is None:
        validation_gap = max(0, int(0.05 * n_time))
    validation_gap = min(validation_gap, int(0.25 * n_time))
    if n_time - forecast_steps - validation_gap < n_lag:
        logging.warning("Adjusting n_lag due to insufficient data for train/validation split.")
        n_lag = max(1, n_time - forecast_steps - validation_gap -1)
    logging.info(f"Window params set: n_lag={n_lag}, forecast_steps={forecast_steps}, validation_gap={validation_gap}")
    return n_lag, forecast_steps, validation_gap, n_time

def build_lstm(input_shape, hyperparams):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(hyperparams.get('lstm_units', 32),
                             activation=hyperparams.get('lstm_activation', 'relu'),
                             input_shape=input_shape),
        tf.keras.layers.Dense(1)
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparams.get('learning_rate', 0.01)),
        loss='mse'
    )
    return model

def build_rf(hyperparams):
    return RandomForestRegressor(
        n_estimators=hyperparams.get('n_estimators', 100),
        random_state=hyperparams.get('rf_random_state', CONFIG["SEED"]),
        max_depth=hyperparams.get('max_depth', None),
        min_samples_split=hyperparams.get('min_samples_split', 2),
        min_samples_leaf=hyperparams.get('min_samples_leaf', 1),
        n_jobs=1
    )

def build_ann(input_shape_flat, hyperparams):
    layers_units = hyperparams.get('ann_layers_units', [64, 32])
    layers = [
        tf.keras.layers.Dense(layers_units[0],
                              activation=hyperparams.get('ann_activation', 'relu'),
                              input_shape=(input_shape_flat,))
    ]
    for units in layers_units[1:]:
        layers.append(tf.keras.layers.Dense(units, activation=hyperparams.get('ann_activation', 'relu')))
    layers.append(tf.keras.layers.Dense(1))
    model = tf.keras.Sequential(layers)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparams.get('learning_rate', 0.01)),
        loss='mse'
    )
    return model

def forecast_arima(ts_train, forecast_steps, max_d=2, return_order=False, return_residual=False):
    y_pred = np.full(forecast_steps, np.nan)
    d_out = np.nan
    residuals = np.full_like(ts_train, np.nan)
    try:
        ts_train_valid = robust_impute(ts_train)
        if len(ts_train_valid) < 10:
            return (y_pred, d_out, residuals) if return_residual else ((y_pred, d_out) if return_order else y_pred)
        auto_model = pm.auto_arima(ts_train_valid,
                                   start_p=1, start_q=1, max_p=3, max_q=3,
                                   d=None, max_d=max_d,
                                   seasonal=False,
                                   stepwise=True, suppress_warnings=True,
                                   error_action='ignore', trace=False)
        y_pred = auto_model.predict(n_periods=forecast_steps)
        y_pred[~np.isfinite(y_pred)] = np.nan
        y_pred = np.maximum(0, y_pred)
        d_out = auto_model.order[1]
        residuals = ts_train_valid - auto_model.predict_in_sample()
    except Exception as e:
        logging.debug(f"ARIMA forecast failed for a pixel: {e}")
    if return_residual:
        return y_pred, d_out, residuals
    if return_order:
        return y_pred, d_out
    return y_pred

def forecast_ml(ts_input_features, target_column_index, n_lag, forecast_steps, model_type, hyperparams, return_residual=False):
    if ts_input_features.ndim != 2 or ts_input_features.shape[0] < n_lag + 1:
        logging.debug(f"Not enough data for ML forecast: shape {ts_input_features.shape}, n_lag {n_lag}")
        if return_residual:
            return np.full(forecast_steps, np.nan), np.full(ts_input_features.shape[0], np.nan)
        return np.full(forecast_steps, np.nan)
    n_features = ts_input_features.shape[1]
    ts_imputed_features = np.array([robust_impute(ts_input_features[:, i]) for i in range(n_features)]).T
    scaler_features = MinMaxScaler()
    ts_scaled_features = scaler_features.fit_transform(ts_imputed_features)
    scaler_target = MinMaxScaler()
    scaler_target.fit(ts_imputed_features[:, target_column_index].reshape(-1, 1))
    X_list, y_list = [], []
    for i in range(len(ts_scaled_features) - n_lag):
        X_list.append(ts_scaled_features[i : i + n_lag, :])
        y_list.append(ts_scaled_features[i + n_lag, target_column_index])
    if not X_list or len(X_list) < 2:
        if return_residual:
            return np.full(forecast_steps, np.nan), np.full(ts_input_features.shape[0], np.nan)
        return np.full(forecast_steps, np.nan)
    X_train = np.array(X_list)
    y_train = np.array(y_list)
    tf.keras.backend.clear_session()
    residuals = np.full(ts_input_features.shape[0], np.nan)
    if model_type == 'lstm':
        model = build_lstm((n_lag, n_features), hyperparams)
        model.fit(X_train, y_train, epochs=hyperparams.get('epochs', 50), batch_size=hyperparams.get('batch_size', 16), verbose=0)
        y_train_pred = model.predict(X_train, verbose=0).flatten()
        residuals[-len(y_train_pred):] = scaler_target.inverse_transform(y_train.reshape(-1, 1)).flatten() - scaler_target.inverse_transform(y_train_pred.reshape(-1, 1)).flatten()
    elif model_type == 'ann':
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        model = build_ann(X_train_flat.shape[1], hyperparams)
        model.fit(X_train_flat, y_train, epochs=hyperparams.get('epochs', 50), batch_size=hyperparams.get('batch_size', 16), verbose=0)
        y_train_pred = model.predict(X_train_flat, verbose=0).flatten()
        residuals[-len(y_train_pred):] = scaler_target.inverse_transform(y_train.reshape(-1, 1)).flatten() - scaler_target.inverse_transform(y_train_pred.reshape(-1, 1)).flatten()
    elif model_type == 'rf':
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        model = build_rf(hyperparams)
        model.fit(X_train_flat, y_train)
        y_train_pred = model.predict(X_train_flat)
        residuals[-len(y_train_pred):] = scaler_target.inverse_transform(y_train.reshape(-1, 1)).flatten() - scaler_target.inverse_transform(y_train_pred.reshape(-1, 1)).flatten()
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
    predictions_scaled = []
    current_sequence_scaled = ts_scaled_features[-n_lag:, :]
    for _ in range(forecast_steps):
        if model_type == 'lstm':
            input_for_pred = current_sequence_scaled.reshape(1, n_lag, n_features)
            pred_scaled = model.predict(input_for_pred, verbose=0)[0, 0]
        elif model_type in ['ann', 'rf']:
            input_for_pred = current_sequence_scaled.flatten().reshape(1, -1)
            if model_type == 'ann':
                pred_scaled = model.predict(input_for_pred, verbose=0)[0, 0]
            else:
                pred_scaled = model.predict(input_for_pred)[0]
        predictions_scaled.append(pred_scaled)
        new_row_scaled = current_sequence_scaled[-1, :].copy()
        new_row_scaled[target_column_index] = pred_scaled
        current_sequence_scaled = np.roll(current_sequence_scaled, -1, axis=0)
        current_sequence_scaled[-1, :] = new_row_scaled
    predictions_unscaled = scaler_target.inverse_transform(np.array(predictions_scaled).reshape(-1, 1)).flatten()
    predictions_unscaled = np.maximum(0, predictions_unscaled)
    if return_residual:
        return predictions_unscaled, residuals
    return predictions_unscaled


2025-05-17 13:45:14,117 INFO:Logging started. All logs saved to D:\DataPenelitian\Longsor\logs\run.log


In [ ]:
# ========== Optuna Hyperparameter Tuning ==========
def optuna_objective(trial, ts_features_pixel, target_col_idx, n_lag, forecast_steps, model_type, validation_gap_tune):
    """
    Fungsi objektif untuk Optuna, bertujuan meminimalkan RMSE pada set validasi kecil.
    ts_features_pixel adalah data fitur untuk satu piksel (time, n_features).
    """
    # Split data untuk tuning: train_tune dan val_tune
    # Pastikan ada cukup data setelah validation_gap_tune dan forecast_steps
    if len(ts_features_pixel) < n_lag + forecast_steps + validation_gap_tune + 1:
        logging.debug(f"Optuna: Not enough data for trial split. Len: {len(ts_features_pixel)}")
        return np.inf # Kembalikan nilai besar jika data tidak cukup

    train_tune_data = ts_features_pixel[:-(forecast_steps + validation_gap_tune)]

    # y_true_val adalah data target aktual untuk periode validasi
    # Indeks awal untuk y_true_val adalah setelah train_tune_data dan validation_gap_tune
    idx_start_val_true = len(train_tune_data) + validation_gap_tune
    y_true_val = ts_features_pixel[idx_start_val_true : idx_start_val_true + forecast_steps, target_col_idx]

    if len(train_tune_data) < n_lag + 1 or len(y_true_val) != forecast_steps :
        logging.debug(f"Optuna: Not enough data after splitting for train_tune or y_true_val. Train_tune len: {len(train_tune_data)}, y_true_val len: {len(y_true_val)}")
        return np.inf

    # Definisikan hyperparameter yang akan di-tune
    if model_type == 'lstm':
        params = {
            "lstm_units": trial.suggest_int("lstm_units", 16, 64, step=8),
            "lstm_activation": trial.suggest_categorical("lstm_activation", ['relu', 'tanh']),
            "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
            "epochs": trial.suggest_int("epochs", 20, 80, step=10),
            "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32]),
        }
    elif model_type == 'ann':
        n_layers = trial.suggest_int("n_layers", 1, 3)
        ann_layers_units = []
        for i in range(n_layers):
            ann_layers_units.append(trial.suggest_int(f"units_layer{i+1}", 16, 128, step=16))
        params = {
            "ann_layers_units": ann_layers_units,
            "ann_activation": trial.suggest_categorical("ann_activation", ['relu', 'tanh']),
            "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
            "epochs": trial.suggest_int("epochs", 20, 80, step=10),
            "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32]),
        }
    elif model_type == 'rf':
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 200, step=25),
            "max_depth": trial.suggest_int("max_depth", 3, 20),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 8),
            "rf_random_state": CONFIG["SEED"], # Jaga random state untuk RF
        }
    else:
        raise ValueError(f"Model type {model_type} not supported for Optuna.")

    try:
        # Lakukan peramalan dengan hyperparameter yang di-trial
        preds_val = forecast_ml(train_tune_data, target_col_idx, n_lag, forecast_steps, model_type, params)

        # Hitung RMSE (atau metrik lain)
        # Pastikan y_true_val dan preds_val memiliki panjang yang sama dan tidak semua NaN
        if len(y_true_val) == len(preds_val) and not np.isnan(y_true_val).all() and not np.isnan(preds_val).all():
            # Imputasi jika ada NaN individual setelah peramalan (jarang terjadi jika input sudah diimputasi)
            y_true_val_imp = robust_impute(y_true_val)
            preds_val_imp = robust_impute(preds_val)
            rmse = np.sqrt(mean_squared_error(y_true_val_imp, preds_val_imp))
        else:
            rmse = np.inf # Kembalikan nilai besar jika prediksi gagal atau tidak valid
    except Exception as e:
        logging.debug(f"Optuna trial failed for {model_type} with params {params}: {e}")
        rmse = np.inf # Jika terjadi error saat training/prediksi
    return rmse


def tune_hyperparameters(ds_pixel_features, target_col_idx, n_lag, forecast_steps, model_type, n_trials=15, validation_gap_tune_ratio=0.1):
    """
    Melakukan tuning hyperparameter menggunakan Optuna untuk satu piksel sampel.
    ds_pixel_features adalah DataArray (time, features) untuk satu piksel.
    """
    logging.info(f"Starting hyperparameter tuning for {model_type} with {n_trials} trials...")

    # Tentukan validation_gap untuk tuning berdasarkan rasio dari panjang data piksel
    # Ini adalah gap antara akhir data training Optuna dan awal periode validasi Optuna
    validation_gap_tune = max(0, int(len(ds_pixel_features) * validation_gap_tune_ratio))

    study = optuna.create_study(direction="minimize") # Tujuan: minimalkan RMSE
    # Fungsi lambda untuk meneruskan argumen tambahan ke objective function
    objective_func = lambda trial: optuna_objective(trial, ds_pixel_features.values, target_col_idx, n_lag, forecast_steps, model_type, validation_gap_tune)

    try:
        study.optimize(objective_func, n_trials=n_trials, show_progress_bar=True)
        logging.info(f"Best {model_type} hyperparams: {study.best_params}, Best RMSE from tuning: {study.best_value:.4f}")
        return study.best_params
    except Exception as e:
        logging.error(f"Optuna study failed for {model_type}: {e}")
        return {} # Kembalikan dict kosong jika tuning gagal total


# ========== Parallel Grid Forecasting ==========
def forecast_grid(ds, model_type, hyperparams, feature_names, target_var, n_lag, forecast_steps, validation_gap, n_jobs, agg_label, return_inter=False):
    """
    Melakukan peramalan untuk seluruh grid secara paralel.

    Args:
        ds (xr.Dataset): Dataset input.
        model_type (str): Tipe model ('arima', 'lstm', 'ann', 'rf').
        hyperparams (dict): Hyperparameter untuk model ML (None untuk ARIMA).
        feature_names (list): Daftar nama fitur yang akan digunakan (termasuk target).
        target_var (str): Nama variabel target.
        n_lag (int): Jumlah lag.
        forecast_steps (int): Jumlah langkah peramalan.
        validation_gap (int): Jarak validasi.
        n_jobs (int): Jumlah job paralel.
        agg_label (str): Label agregasi waktu.
        return_inter (bool): Jika True dan model ARIMA, kembalikan juga peta orde differencing.

    Returns:
        np.ndarray or tuple: Array prediksi (forecast_steps, y, x), atau (prediksi, order_diff_map) untuk ARIMA.
    """
    y_dim, x_dim = ds[target_var].shape[1:]
    results = np.full((forecast_steps, y_dim, x_dim), np.nan) # Inisialisasi array hasil prediksi
    order_diff_map = np.full((y_dim, x_dim), np.nan) if model_type == "arima" else None

    # Tentukan indeks kolom target
    try:
        target_column_index = feature_names.index(target_var)
    except ValueError:
        logging.error(f"Target variable '{target_var}' not found in feature_names: {feature_names}. Cannot proceed with ML model.")
        if return_inter and model_type == "arima": return results, order_diff_map
        return results

    # Fungsi untuk memproses satu piksel
    def single_pixel(i, j):
        # Ambil data time series untuk semua fitur pada piksel (i,j)
        # ds[feature_names] akan menghasilkan Dataset, .to_array() mengubahnya jadi DataArray (variable, time, y, x)
        # lalu kita select y=i, x=j, dan transpose agar menjadi (time, variable/features)
        ts_all_features_pixel = ds[feature_names].isel(y=i, x=j).to_array(dim="feature_dim").transpose("time", "feature_dim").values

        # Data training adalah semua data sebelum periode validasi dan peramalan
        # (forecast_steps + validation_gap) adalah panjang periode yang tidak digunakan untuk training
        split_idx = len(ts_all_features_pixel) - (forecast_steps + validation_gap)
        if split_idx <= 0: # Tidak cukup data untuk training
            logging.debug(f"Pixel ({i},{j}): Not enough data for train split. Total len: {len(ts_all_features_pixel)}, split_idx: {split_idx}")
            return (np.full(forecast_steps, np.nan), np.nan) if model_type == "arima" and return_inter else np.full(forecast_steps, np.nan)

        ts_train_features = ts_all_features_pixel[:split_idx, :]

        if model_type == "arima":
            # ARIMA hanya menggunakan variabel target untuk training
            ts_train_target_only = ts_train_features[:, target_column_index]
            preds, d_val = forecast_arima(ts_train_target_only, forecast_steps, return_order=True)
            return preds, d_val
        else: # Model ML
            # Pastikan ada cukup data training untuk membentuk lag
            if len(ts_train_features) < n_lag + 1:
                 logging.debug(f"Pixel ({i},{j}) for {model_type}: Not enough training data ({len(ts_train_features)}) for n_lag ({n_lag}).")
                 return np.full(forecast_steps, np.nan)

            preds = forecast_ml(ts_train_features, target_column_index, n_lag, forecast_steps, model_type, hyperparams)
            return preds

    all_pixels = [(i, j) for i in range(y_dim) for j in range(x_dim)] # Daftar semua koordinat piksel

    # Jalankan peramalan secara paralel
    if model_type == "arima":
        # Untuk ARIMA, single_pixel mengembalikan (preds, d_val)
        pred_and_order = Parallel(n_jobs=n_jobs)(
            delayed(single_pixel)(i, j) for i, j in tqdm(all_pixels, desc=f"Forecast grid {model_type} [{agg_label}]")
        )
        for idx, (i, j) in enumerate(all_pixels):
            preds, d_val = pred_and_order[idx]
            results[:, i, j] = preds
            if order_diff_map is not None: order_diff_map[i, j] = d_val
        if return_inter:
            return results, order_diff_map
        return results
    else: # Model ML
        # Untuk ML, single_pixel hanya mengembalikan preds
        pred_list = Parallel(n_jobs=n_jobs)(
            delayed(single_pixel)(i, j) for i, j in tqdm(all_pixels, desc=f"Forecast grid {model_type} [{agg_label}]")
        )
        for idx, (i, j) in enumerate(all_pixels):
            results[:, i, j] = pred_list[idx]
        if return_inter: # Walaupun untuk ML tidak ada order_diff_map, jaga konsistensi return
            return results, None
        return results

# ========== Evaluation ==========
def evaluate_grid(ds, preds, target_var, forecast_steps, validation_gap):
    """
    Mengevaluasi prediksi grid terhadap data observasi.

    Args:
        ds (xr.Dataset): Dataset input (mengandung data observasi).
        preds (np.ndarray): Array prediksi (forecast_steps, y, x).
        target_var (str): Nama variabel target.
        forecast_steps (int): Jumlah langkah peramalan.
        validation_gap (int): Jarak validasi.

    Returns:
        tuple: r2_map, rmse_map, mae_map (semua berbentuk array 2D (y,x)).
    """
    y_dim, x_dim = ds[target_var].shape[1:]
    r2_map = np.full((y_dim, x_dim), np.nan)
    rmse_map = np.full((y_dim, x_dim), np.nan)
    mae_map = np.full((y_dim, x_dim), np.nan)

    # Tentukan indeks waktu untuk data observasi yang akan dibandingkan
    # Data observasi dimulai setelah periode training dan validation_gap
    # dan berakhir setelah forecast_steps
    n_time_total = ds[target_var].shape[0]
    idx_start_true = n_time_total - forecast_steps # Indeks awal data observasi untuk perbandingan
    idx_end_true = n_time_total # Indeks akhir (eksklusif)

    if validation_gap > 0 : # Jika ada gap, data observasi ada setelah gap tersebut
        idx_start_true = n_time_total - forecast_steps - validation_gap + validation_gap # Ini salah, harusnya:
        idx_start_true = n_time_total - forecast_steps # Prediksi dibandingkan dengan data aktual *setelah* gap
        # Misal: T=100, FS=10, VG=5. Train: 0-84. Pred: 85-94. True: 95-99 (Ini jika VG adalah antara train dan test)
        # Jika VG adalah antara akhir data dan awal forecast:
        # Train: 0-(T-FS-VG-1). Forecast: (T-FS-VG) -> (T-VG-1). True: (T-FS) -> (T-1)
        # Dalam skrip ini, VG adalah antara akhir training dan awal periode validasi/peramalan.
        # Jadi, y_true adalah data aktual pada periode peramalan.
        idx_start_true = n_time_total - forecast_steps
        idx_end_true = n_time_total


    if idx_start_true < 0 or idx_end_true > n_time_total or idx_start_true >= idx_end_true :
        logging.error(f"Invalid time indices for evaluation. Start: {idx_start_true}, End: {idx_end_true}, Total: {n_time_total}")
        return r2_map, rmse_map, mae_map

    y_true_all_pixels = ds[target_var][idx_start_true:idx_end_true, :, :].values # (forecast_steps, y, x)

    for i in range(y_dim):
        for j in range(x_dim):
            y_true_pixel = y_true_all_pixels[:, i, j] # (forecast_steps,)
            y_pred_pixel = preds[:, i, j] # (forecast_steps,)

            if np.isnan(y_true_pixel).all() or np.isnan(y_pred_pixel).all():
                continue # Lewati jika semua observasi atau prediksi adalah NaN

            # Imputasi untuk kasus NaN sporadis (seharusnya sudah ditangani sebelumnya)
            y_true_imp = robust_impute(y_true_pixel)
            y_pred_imp = robust_impute(y_pred_pixel)

            # Hanya hitung metrik jika ada lebih dari 1 titik data valid dan ada varians
            if len(y_true_imp[~np.isnan(y_true_imp)]) > 1 and np.std(y_true_imp[~np.isnan(y_true_imp)]) > 1e-6 :
                try:
                    r2_map[i, j] = r2_score(y_true_imp, y_pred_imp)
                except ValueError: pass # Terjadi jika y_true konstan
            else: # Jika data konstan atau hanya 1 titik, R2 tidak terdefinisi atau 0
                 if len(y_true_imp[~np.isnan(y_true_imp)]) > 0 and len(y_pred_imp[~np.isnan(y_pred_imp)]) > 0:
                     if np.allclose(y_true_imp, y_pred_imp): r2_map[i,j] = 1.0
                     else: r2_map[i,j] = 0.0 # Atau NaN, tergantung definisi yang diinginkan

            if len(y_true_imp[~np.isnan(y_true_imp)]) > 0: # Cukup ada data untuk RMSE dan MAE
                rmse_map[i, j] = np.sqrt(mean_squared_error(y_true_imp, y_pred_imp))
                mae_map[i, j] = mean_absolute_error(y_true_imp, y_pred_imp)

    return r2_map, rmse_map, mae_map

# ========== Diagnostic: Ljung-Box ==========
def ljungbox_pvals(ts_residuals, lags=10, model_df=0):
    """
    Menghitung p-value dari uji Ljung-Box pada residual.

    Args:
        ts_residuals (np.ndarray): Time series residual.
        lags (int): Jumlah lag untuk diuji.
        model_df (int): Jumlah parameter model yang diestimasi (untuk penyesuaian df).

    Returns:
        float: p-value, atau NaN jika gagal.
    """
    try:
        import statsmodels.api as sm # Impor di dalam fungsi untuk menghindari error jika tidak terinstall global
        ts_residuals_clean = robust_impute(ts_residuals) # Bersihkan residual
        ts_residuals_clean = ts_residuals_clean[~np.isnan(ts_residuals_clean)] # Hapus NaN sisa

        if len(ts_residuals_clean) <= lags or len(ts_residuals_clean) < 2 or np.std(ts_residuals_clean) < 1e-9:
            return np.nan # Tidak cukup data atau tidak ada varians

        # Pastikan lags tidak melebihi jumlah observasi - model_df - 1
        actual_lags = min(lags, len(ts_residuals_clean) - model_df - 1)
        if actual_lags <= 0: return np.nan

        lb_test_df = sm.stats.acorr_ljungbox(ts_residuals_clean, lags=[actual_lags], return_df=True, model_df=model_df)
        return lb_test_df['lb_pvalue'].values[0] if not lb_test_df.empty else np.nan
    except Exception as e:
        logging.debug(f"Ljung-Box test failed: {e}")
        return np.nan

def diagnostic_ljungbox_grid(ds_train_residuals_map, lags=10, model_df_map=None):
    """
    Melakukan uji Ljung-Box pada residual training untuk seluruh grid.
    ds_train_residuals_map: dict, model -> np.ndarray (y,x,time_residuals)
    model_df_map: dict, model -> int (jumlah parameter, misal p+q untuk ARIMA)
    """
    if not ds_train_residuals_map: return {}

    pval_maps_dict = {}

    first_model = list(ds_train_residuals_map.keys())[0]
    y_dim, x_dim, _ = ds_train_residuals_map[first_model].shape # (y, x, time_residuals)

    for model_name, residuals_grid in ds_train_residuals_map.items():
        pval_map = np.full((y_dim, x_dim), np.nan)
        current_model_df = model_df_map.get(model_name, 0) if model_df_map else 0
        for i in range(y_dim):
            for j in range(x_dim):
                ts_resid_pixel = residuals_grid[i, j, :]
                pval_map[i, j] = ljungbox_pvals(ts_resid_pixel, lags=lags, model_df=current_model_df)
        pval_maps_dict[model_name] = pval_map
        logging.info(f"Ljung-Box p-value map calculated for {model_name}.")
    return pval_maps_dict


# ========== Output: GeoTIFF, NetCDF, PNG, GIF ==========
def ensure_extension(filename, ext):
    if not ext.startswith('.'):
        ext = '.' + ext
    if not filename.lower().endswith(ext):
        filename += ext
    return filename

def hash_params(params: dict) -> str:
    if not params: return "default"
    s = str(sorted(params.items()))
    return hashlib.md5(s.encode()).hexdigest()[:8]

def setup_folders(base_dir, folders):
    out_dirs = {}
    for k, v in folders.items():
        d = os.path.join(base_dir, v)
        os.makedirs(d, exist_ok=True)
        out_dirs[k] = d
    return out_dirs
OUTDIRS = setup_folders(CONFIG["BASE_DIR"], OUTPUT_FOLDERS)

def setup_logging(log_dir):
    log_path = os.path.join(log_dir, "run.log")
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s:%(message)s',
        handlers=[logging.FileHandler(log_path, mode='w'), logging.StreamHandler()]
    )
    logging.info("Logging started. All logs saved to %s", log_path)
setup_logging(OUTDIRS["logs"])

def robust_impute(ts):
    s = pd.Series(ts)
    s = s.interpolate(method='linear', limit_direction='both').fillna(method='bfill').fillna(method='ffill').fillna(0)
    return s.values

def fallback_crs(ds, config):
    try:
        if not hasattr(ds, 'rio') or ds.rio.crs is None:
            ds = ds.rio.write_crs(config["TARGET_CRS"])
            logging.info(f"Fallback CRS applied: {config['TARGET_CRS']}")
    except Exception as e:
        logging.warning(f"Fallback CRS failed: {e}")
    return ds

def calculate_spatial_avg_lag1(data_array, feature_name="SPATIAL_AVG_LAG1"):
    kernel = np.array([[1,1,1],[1,0,1],[1,1,1]], dtype=np.float32)
    values = data_array.values.astype(np.float32)
    spatial_avg_lag1 = np.full_like(values, np.nan)
    for t in range(1, values.shape[0]):
        data_t_minus_1 = values[t-1,:,:]
        nan_mask_t_minus_1 = np.isnan(data_t_minus_1)
        neighbor_counts = convolve(~nan_mask_t_minus_1, kernel, mode='constant', cval=0.0)
        data_t_minus_1_nan_as_zero = np.nan_to_num(data_t_minus_1, nan=0.0)
        neighbor_sum = convolve(data_t_minus_1_nan_as_zero, kernel, mode='constant', cval=0.0)
        valid_neighbors_mask = neighbor_counts > 0
        avg_values = np.full_like(neighbor_sum, np.nan)
        avg_values[valid_neighbors_mask] = neighbor_sum[valid_neighbors_mask] / neighbor_counts[valid_neighbors_mask]
        spatial_avg_lag1[t,:,:] = avg_values
    spatial_da = xr.DataArray(
        spatial_avg_lag1, coords=data_array.coords, dims=data_array.dims, name=feature_name,
        attrs={'long_name': f'Average of 8 spatial neighbors at lag 1 for {data_array.name}',
               'units': data_array.attrs.get('units', 'unknown')}
    )
    logging.info(f"Calculated spatial feature: {feature_name}")
    return spatial_da

def add_time_features(ds, time_var='time'):
    try:
        months = pd.to_datetime(ds[time_var].values).month
        ds['month'] = (ds[time_var].dims, months)
        ds['sin_month'] = (ds[time_var].dims, np.sin(2 * np.pi * months / 12))
        ds['cos_month'] = (ds[time_var].dims, np.cos(2 * np.pi * months / 12))
        logging.info("Added time features: month, sin_month, cos_month.")
    except Exception as e:
        logging.warning(f"Could not add time features: {e}")
    return ds

def detect_agg_label(ds):
    time = pd.to_datetime(ds['time'].values)
    if len(time) < 2:
        return "1m"
    delta = (time[1] - time[0]).days
    if 25 <= delta < 35:
        return "1m"
    elif 55 <= delta < 65:
        return "2m"
    elif 85 <= delta < 95:
        return "3m"
    else:
        return f"{delta}d"

def forecast_filename(model, agg_label, start_date, end_date, step_ext):
    return f"{model}_forecast_{agg_label}_{start_date}_to_{end_date}_{step_ext}"

def evalmap_filename(model, agg_label, metric, ext):
    return f"{model}_eval_{metric}_{agg_label}.{ext}"

def difforder_filename(model, agg_label, ext):
    return f"{model}_orderdiff_{agg_label}.{ext}"

def save_geotiff(arr, ds_ref, outdir, fname, descr=None, nodata_val=np.nan):
    fname = ensure_extension(fname, "tif")
    coords = {"y": ds_ref["y"], "x": ds_ref["x"]}
    da = xr.DataArray(arr, dims=("y", "x"), coords=coords)
    if hasattr(ds_ref, 'rio') and ds_ref.rio.crs:
        da = da.rio.write_crs(ds_ref.rio.crs, inplace=True)
    if hasattr(ds_ref, 'rio') and ds_ref.rio.transform():
        if not ds_ref.rio.transform().is_identity:
             da = da.rio.write_transform(ds_ref.rio.transform(), inplace=True)
    if descr:
        da.attrs['long_name'] = descr
    try:
        da = da.rio.set_spatial_dims(x_dim='x', y_dim='y', inplace=True)
    except Exception:
        pass
    path = os.path.join(outdir, fname)
    try:
        da.rio.to_raster(path, nodata=nodata_val, tiled=True, compress='LZW', num_threads='ALL_CPUS')
        logging.info(f"Saved GeoTIFF: {path}")
    except Exception as e:
        logging.error(f"Failed to save GeoTIFF {path}: {e}")

def save_gif(preds_stack, time_labels, outdir, base_fname, title_prefix="", vmin=None, vmax=None, fps=2):
    base_fname = ensure_extension(base_fname, "gif")
    imgs = []
    if vmin is None: vmin = np.nanmin(preds_stack)
    if vmax is None: vmax = np.nanmax(preds_stack)
    if np.isnan(vmin): vmin = 0
    if np.isnan(vmax): vmax = 1
    if vmin == vmax : vmax = vmin + 1
    temp_dir_frames = os.path.join(outdir, "temp_frames_for_gif")
    os.makedirs(temp_dir_frames, exist_ok=True)
    for t in range(preds_stack.shape[0]):
        fig, ax = plt.subplots(figsize=(7,6))
        frame_data = np.nan_to_num(preds_stack[t], nan=vmin-abs(vmin*0.1+1))
        im = ax.imshow(frame_data, vmin=vmin, vmax=vmax, cmap=CONFIG["COLORMAP"], origin="upper")
        date_str = pd.to_datetime(time_labels[t]).strftime('%Y-%m-%d')
        ax.set_title(f"{title_prefix} Forecast: {date_str}", fontsize=10)
        ax.set_xlabel("X-coordinate index")
        ax.set_ylabel("Y-coordinate index")
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label="Predicted Value")
        tmp_path = os.path.join(temp_dir_frames, f"frame_{t:03d}.png")
        plt.savefig(tmp_path, bbox_inches="tight", dpi=100)
        plt.close(fig)
        imgs.append(imageio.v2.imread(tmp_path))
    gif_path = os.path.join(outdir, base_fname)
    try:
        imageio.mimsave(gif_path, imgs, fps=fps, loop=0)
        logging.info(f"Saved GIF: {gif_path}")
    except Exception as e:
        logging.error(f"Failed to save GIF {gif_path}: {e}")
    finally:
        for t in range(preds_stack.shape[0]):
            tmp_path = os.path.join(temp_dir_frames, f"frame_{t:03d}.png")
            if os.path.exists(tmp_path):
                os.remove(tmp_path)
        if os.path.exists(temp_dir_frames):
            os.rmdir(temp_dir_frames)

def save_eval_map_png_and_geotiff(arr, ds_ref, outdir_png, outdir_geotiff, base_fname, title, cmap="viridis", nodata_val=np.nan):
    png_fname = ensure_extension(base_fname, "png")
    tif_fname = ensure_extension(base_fname, "tif")
    save_geotiff(arr, ds_ref, outdir_geotiff, tif_fname, descr=title, nodata_val=nodata_val)
    plt.figure(figsize=(8, 6))
    valid_arr = arr[~np.isnan(arr)]
    vmin_plot, vmax_plot = (np.min(valid_arr), np.max(valid_arr)) if len(valid_arr) > 0 else (0,1)
    if vmin_plot == vmax_plot and len(valid_arr)>0 : vmax_plot = vmin_plot + 1
    if not len(valid_arr)>0 : vmin_plot, vmax_plot = 0,1
    im = plt.imshow(arr, origin="upper", cmap=cmap, vmin=vmin_plot, vmax=vmax_plot)
    plt.title(title, fontsize=12)
    plt.xlabel("X-coordinate index")
    plt.ylabel("Y-coordinate index")
    plt.colorbar(im, label="Metric Value")
    plt.tight_layout()
    png_path = os.path.join(outdir_png, png_fname)
    try:
        plt.savefig(png_path, dpi=150)
        logging.info(f"Saved PNG map: {png_path}")
    except Exception as e:
        logging.error(f"Failed to save PNG map {png_path}: {e}")
    plt.close()

def save_netcdf_output(ds_out, outdir, fname_base, agg_label, time_labels_forecast):
    fname = ensure_extension(fname_base, "nc")
    path = os.path.join(outdir, fname)
    encoding = {}
    for var_name in ds_out.data_vars:
        encoding[var_name] = {'_FillValue': np.nan, 'zlib': True, 'complevel': 4}
    for coord_name in ds_out.coords:
        if ds_out[coord_name].dtype.kind in 'ifc':
             encoding[coord_name] = {'_FillValue': None}
        elif ds_out[coord_name].dtype.kind == 'M':
             encoding[coord_name] = {'_FillValue': None, 'dtype': 'double', 'units': "days since 1970-01-01"}
    try:
        ds_out.to_netcdf(path, encoding=encoding, format='NETCDF4')
        logging.info(f"Saved NetCDF: {path}")
    except Exception as e:
        logging.error(f"Failed to save NetCDF {path}: {e}")


# ========== Plotting: Time Series Agregat ==========
def plot_ts_aggregate(ds, preds_dict, target_var, forecast_steps, validation_gap, outdir, n_time, agg_label, time_index_all):
    """
    Membuat plot time series agregat (total nilai target di seluruh grid).

    Args:
        ds (xr.Dataset): Dataset input.
        preds_dict (dict): Dictionary prediksi {model_name: preds_array}.
        target_var (str): Nama variabel target.
        forecast_steps (int): Jumlah langkah peramalan.
        validation_gap (int): Jarak validasi.
        outdir (str): Direktori output untuk plot.
        n_time (int): Jumlah total langkah waktu.
        agg_label (str): Label agregasi waktu.
        time_index_all (pd.DatetimeIndex): Indeks waktu untuk seluruh periode.
    """
    obs = ds[target_var].values # (time, y, x)
    obs_total = np.nansum(obs, axis=(1,2)) # Agregasi spasial (sum)

    # Indeks waktu di mana peramalan dimulai pada data observasi
    # Ini adalah setelah periode training dan validation_gap
    idx_forecast_start_on_obs = n_time - forecast_steps

    plt.figure(figsize=(14, 7)) # Ukuran disesuaikan

    # Plot data observasi keseluruhan
    plt.plot(time_index_all, obs_total, label="Observed (Total Grid)", color="gray", linewidth=1.5, alpha=0.7)

    # Garis vertikal menandai awal periode peramalan
    if idx_forecast_start_on_obs < len(time_index_all) and idx_forecast_start_on_obs >=0:
        plt.axvline(time_index_all[idx_forecast_start_on_obs], color="black", linestyle="--", linewidth=1.5, label=f"Forecast/Validation Start ({time_index_all[idx_forecast_start_on_obs].strftime('%Y-%m')})")

    # Indeks waktu untuk plot prediksi dan observasi pada periode validasi/peramalan
    time_index_forecast_period = time_index_all[idx_forecast_start_on_obs : idx_forecast_start_on_obs + forecast_steps]

    # Plot prediksi dari setiap model
    colors = plt.cm.get_cmap('tab10', len(preds_dict)) # Palet warna
    for i, (model, preds_model) in enumerate(preds_dict.items()):
        preds_total = np.nansum(preds_model, axis=(1,2)) # Agregasi spasial prediksi
        if len(preds_total) == len(time_index_forecast_period):
            plt.plot(time_index_forecast_period, preds_total, marker='o', linestyle='-', markersize=5, label=f"{model.upper()} Forecast", color=colors(i))
        else:
            logging.warning(f"Length mismatch for {model} aggregate plot. Preds: {len(preds_total)}, Time: {len(time_index_forecast_period)}")


    # Plot data observasi pada periode training dan validasi (jika ada)
    if idx_forecast_start_on_obs > 0: # Jika ada periode training
        plt.plot(time_index_all[:idx_forecast_start_on_obs], obs_total[:idx_forecast_start_on_obs], color="blue", linestyle='-', marker='.', markersize=4, label="Training Period (Observed)")

    # Plot data observasi pada periode validasi (yang dibandingkan dengan forecast)
    obs_validation_period = obs_total[idx_forecast_start_on_obs : idx_forecast_start_on_obs + forecast_steps]
    if len(obs_validation_period) == len(time_index_forecast_period):
        plt.plot(time_index_forecast_period, obs_validation_period, color="red", linestyle='-', marker='x', markersize=5, label="Validation Period (Observed)")

    plt.legend(loc="best", fontsize=10)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel(f"Total {target_var} (Aggregated over Grid)", fontsize=12)
    plt.title(f"Aggregated Time Series Forecast vs. Observation [{agg_label}]", fontsize=14)
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.tight_layout()

    fname = f"aggregate_timeseries_plot_{agg_label}.png"
    path = os.path.join(outdir, fname)
    try:
        plt.savefig(path, dpi=150)
        logging.info(f"Saved aggregate time series plot: {path}")
    except Exception as e:
        logging.error(f"Failed to save aggregate plot {path}: {e}")
    plt.close()

2025-05-17 13:45:14,190 INFO:Logging started. All logs saved to D:\DataPenelitian\Longsor\logs\run.log


In [ ]:
# ========== MAIN ==========
def main():
    logging.info("===== PIPELINE STARTED =====")
    try:
        # Set working dir
        if os.path.exists(CONFIG["BASE_DIR"]):
            os.chdir(CONFIG["BASE_DIR"])
            logging.info(f"Changed working directory to: {CONFIG['BASE_DIR']}")
        else:
            logging.error(f"BASE_DIR {CONFIG['BASE_DIR']} does not exist. Exiting.")
            return

        # Load Data
        logging.info(f"Loading NetCDF data from: {CONFIG['NC_FILE']}")
        ds = xr.open_dataset(CONFIG["NC_FILE"])
        ds = fallback_crs(ds, CONFIG) # Pastikan CRS ada
        agg_label = detect_agg_label(ds) # Deteksi label agregasi waktu
        time_index_all = pd.to_datetime(ds['time'].values) # Indeks waktu keseluruhan

        # Preprocessing & Feature Engineering
        logging.info("Starting preprocessing and feature engineering...")
        # 1. Proses data longsor jika ada
        if CONFIG["LANDSLIDE_SHP_FILE"] and os.path.exists(CONFIG["LANDSLIDE_SHP_FILE"]):
            landslide_da = process_landslide_data(CONFIG["LANDSLIDE_SHP_FILE"], CONFIG["LANDSLIDE_DATE_COLUMN"], ds, CONFIG["LANDSLIDE_FEATURE_NAME"])
            if landslide_da is not None:
                ds[CONFIG["LANDSLIDE_FEATURE_NAME"]] = landslide_da
        else:
            logging.info(f"Landslide SHP file not found or not specified. Skipping landslide feature.")
            # Buat array nol jika tidak ada data longsor, agar struktur fitur konsisten
            ds[CONFIG["LANDSLIDE_FEATURE_NAME"]] = xr.DataArray(
                np.zeros((ds.dims['time'], ds.dims['y'], ds.dims['x']), dtype=np.int32),
                coords=ds[CONFIG["TARGET_VARIABLE"]].coords, dims=ds[CONFIG["TARGET_VARIABLE"]].dims
            )


        # 2. Hitung fitur spasial
        ds[CONFIG["SPATIAL_FEATURE_NAME"]] = calculate_spatial_avg_lag1(ds[CONFIG["TARGET_VARIABLE"]], CONFIG["SPATIAL_FEATURE_NAME"])

        # 3. Tambah fitur waktu (sin/cos bulan)
        ds = add_time_features(ds) # Menambahkan 'month', 'sin_month', 'cos_month'

        # Daftar fitur yang akan digunakan oleh model ML
        # ARIMA hanya akan menggunakan TARGET_VARIABLE
        feature_names_for_ml = [
            CONFIG["TARGET_VARIABLE"],
            CONFIG["LANDSLIDE_FEATURE_NAME"],
            CONFIG["SPATIAL_FEATURE_NAME"],
            'sin_month',
            'cos_month'
        ]
        # Pastikan semua fitur ada di dataset, jika tidak, hapus dari daftar
        feature_names_for_ml = [f for f in feature_names_for_ml if f in ds]
        logging.info(f"Features for ML models: {feature_names_for_ml}")
        if CONFIG["TARGET_VARIABLE"] not in feature_names_for_ml:
            logging.error(f"Target variable {CONFIG['TARGET_VARIABLE']} is missing after feature engineering. Exiting.")
            return
        target_column_idx_ml = feature_names_for_ml.index(CONFIG["TARGET_VARIABLE"])


        # Tentukan parameter jendela (lag, forecast steps, validation gap)
        n_lag, forecast_steps, validation_gap, n_time = set_window_params(ds, CONFIG)
        logging.info(f"Final window params: n_lag={n_lag}, forecast_steps={forecast_steps}, validation_gap={validation_gap}, n_time_total={n_time}")

        # Inisialisasi dictionary untuk menyimpan prediksi dan hyperparameter terbaik
        all_model_predictions = {} # {model_name: prediction_array (steps, y, x)}
        all_model_eval_metrics = {} # {model_name: {metric_name: metric_map (y,x)}}
        all_model_residuals_train = {} # {model_name: residuals_array_train (y,x,time)}
        best_hyperparams_all_models = {}
        arima_order_diff_map = None # Khusus untuk ARIMA

        # Hyperparameter tuning (jika diaktifkan)
        if CONFIG["HYPERPARAM_TUNE"]:
            logging.info("--- Starting Hyperparameter Tuning ---")
            # Ambil data dari piksel sampel untuk tuning
            # Gunakan semua fitur yang relevan untuk piksel sampel tersebut
            sample_y, sample_x = CONFIG["SAMPLE_COORDS"][0]
            ds_sample_pixel_features = ds[feature_names_for_ml].isel(y=sample_y, x=sample_x).to_array(dim="feature_dim").transpose("time", "feature_dim")

            for model_name in MODEL_LIST:
                if model_name != "arima": # ARIMA tidak di-tune dengan Optuna di sini
                    best_hyperparams_all_models[model_name] = tune_hyperparameters(
                        ds_sample_pixel_features, target_column_idx_ml, n_lag, forecast_steps, model_name,
                        CONFIG["OPTUNA_TRIALS"]
                    )
                else:
                    best_hyperparams_all_models[model_name] = {} # Tidak ada hyperparams eksternal untuk ARIMA auto
            logging.info("--- Hyperparameter Tuning Finished ---")
        else:
            logging.info("Skipping hyperparameter tuning. Using default or pre-defined parameters.")
            # Isi dengan hyperparameter default jika tidak tuning (atau biarkan kosong jika model punya default sendiri)
            for model_name in MODEL_LIST: best_hyperparams_all_models[model_name] = {}


        # Forecast grid per model
        logging.info("--- Starting Grid Forecasting ---")
        for model_name in MODEL_LIST:
            logging.info(f"Processing model: {model_name.upper()}")
            current_hyperparams = best_hyperparams_all_models.get(model_name, {}) # Ambil hyperparams hasil tuning atau default

            if model_name == "arima":
                # ARIMA hanya menggunakan target variable, jadi feature_names tidak relevan untuk forecast_grid-nya
                preds_grid, arima_order_diff_map_current = forecast_grid(
                    ds, model_name, None, [CONFIG["TARGET_VARIABLE"]], CONFIG["TARGET_VARIABLE"],
                    n_lag, forecast_steps, validation_gap, CONFIG["N_JOBS_PARALLEL"], agg_label, return_inter=True
                )
                arima_order_diff_map = arima_order_diff_map_current # Simpan peta orde 'd'
            else: # Model ML
                preds_grid, _ = forecast_grid( # _ untuk order_diff_map yang None untuk ML
                    ds, model_name, current_hyperparams, feature_names_for_ml, CONFIG["TARGET_VARIABLE"],
                    n_lag, forecast_steps, validation_gap, CONFIG["N_JOBS_PARALLEL"], agg_label, return_inter=True
                )
            all_model_predictions[model_name] = preds_grid
            logging.info(f"Grid forecast for {model_name} complete.")

            # Simpan prediksi GeoTIFF per langkah waktu
            # Tentukan tanggal mulai dan akhir periode peramalan
            # Periode peramalan dimulai setelah data training dan validation_gap
            idx_forecast_period_start = n_time - forecast_steps
            time_labels_forecast = time_index_all[idx_forecast_period_start : idx_forecast_period_start + forecast_steps]

            if len(time_labels_forecast) == preds_grid.shape[0]:
                for t_step in range(preds_grid.shape[0]):
                    fname_geotiff_step = forecast_filename(model_name, agg_label, time_labels_forecast[t_step].strftime("%Y%m%d"), time_labels_forecast[t_step].strftime("%Y%m%d"), f"step{t_step+1}.tif")
                    save_geotiff(preds_grid[t_step, :, :], ds, OUTDIRS["geotiff"], fname_geotiff_step,
                                 descr=f"{model_name.upper()} Forecast Step {t_step+1} ({time_labels_forecast[t_step].strftime('%Y-%m-%d')}) [{agg_label}]")
                # Simpan animasi GIF
                gif_fname_base = forecast_filename(model_name, agg_label, time_labels_forecast[0].strftime("%Y%m%d"), time_labels_forecast[-1].strftime("%Y%m%d"), "gif")
                save_gif(preds_grid, time_labels_forecast, OUTDIRS["gif"], gif_fname_base, title_prefix=f"{model_name.upper()}", fps=CONFIG["GIF_FPS"])
            else:
                logging.warning(f"Skipping GeoTIFF/GIF saving for {model_name} due to time label mismatch.")

            # Simpan prediksi mentah (npz) untuk analisis lebih lanjut jika perlu
            npz_fname_base = forecast_filename(model_name, agg_label, time_labels_forecast[0].strftime("%Y%m%d"), time_labels_forecast[-1].strftime("%Y%m%d"), "npz")
            np.savez_compressed(os.path.join(OUTDIRS["intermediate"], npz_fname_base), preds=preds_grid, allow_pickle=False)
            logging.info(f"Saved raw predictions (npz) for {model_name}.")

        # Simpan peta orde differencing ARIMA jika ada
        if arima_order_diff_map is not None:
            diff_fname_base = difforder_filename("arima", agg_label, "")[:-1] # Hapus titik terakhir
            save_eval_map_png_and_geotiff(arima_order_diff_map, ds, OUTDIRS["png"], OUTDIRS["geotiff"],
                                          diff_fname_base, f"ARIMA Differencing Order (d) [{agg_label}]", cmap="coolwarm")

        # Evaluasi Model
        logging.info("--- Starting Model Evaluation ---")
        for model_name, preds_model in all_model_predictions.items():
            r2_map, rmse_map, mae_map = evaluate_grid(ds, preds_model, CONFIG["TARGET_VARIABLE"], forecast_steps, validation_gap)
            all_model_eval_metrics[model_name] = {"r2": r2_map, "rmse": rmse_map, "mae": mae_map}
            logging.info(f"Evaluation metrics calculated for {model_name}.")
            for metric_name, metric_map_data in all_model_eval_metrics[model_name].items():
                eval_fname_base = evalmap_filename(model_name, agg_label, metric_name, "")[:-1]
                save_eval_map_png_and_geotiff(metric_map_data, ds, OUTDIRS["png"], OUTDIRS["geotiff"],
                                              eval_fname_base, f"{model_name.upper()} - {metric_name.upper()} Map [{agg_label}]",
                                              cmap='viridis' if metric_name != 'r2' else 'viridis_r') # r2 lebih baik jika dibalik

        # Diagnostik Residual (Ljung-Box pada residual training)
        # Ini memerlukan penyimpanan residual training dari `forecast_grid` atau menjalankannya lagi.
        # Untuk saat ini, kita akan skip bagian ini karena `forecast_grid` tidak mengembalikan residual.
        # Jika ingin diimplementasikan, `single_pixel` perlu dimodifikasi untuk mengembalikan residual training.
        logging.info("Skipping Ljung-Box on training residuals for now (requires modification to return residuals).")
        # pval_maps_all_models = diagnostic_ljungbox_grid(all_model_residuals_train, ...)
        # for model_name, pval_map_model in pval_maps_all_models.items():
        #     diag_fname_base = residualdiag_filename(model_name, agg_label, "")[:-1]
        #     save_eval_map_png_and_geotiff(pval_map_model, ds, OUTDIRS["png"], OUTDIRS["geotiff"],
        #                                   diag_fname_base, f"{model_name.upper()} Ljung-Box p-value (Train Resid) [{agg_label}]", cmap='coolwarm_r')


        # Membuat Dataset Output NetCDF Gabungan
        logging.info("--- Creating Combined Output NetCDF ---")
        ds_output_combined = xr.Dataset(
            coords={'time_forecast': time_labels_forecast,
                    'y': ds['y'],
                    'x': ds['x']}
        )
        # Tambahkan prediksi dari setiap model
        for model_name, preds_array in all_model_predictions.items():
            da_pred = xr.DataArray(
                preds_array,
                coords={'time_forecast': time_labels_forecast, 'y': ds['y'], 'x': ds['x']},
                dims=('time_forecast', 'y', 'x'),
                name=f'{model_name}_prediction'
            )
            da_pred.attrs['long_name'] = f'{model_name.upper()} {forecast_steps}-step ahead forecast'
            da_pred.attrs['units'] = ds[CONFIG["TARGET_VARIABLE"]].attrs.get('units', 'unknown')
            ds_output_combined[f'{model_name}_prediction'] = da_pred

        # Tambahkan metrik evaluasi
        for model_name, metrics_dict in all_model_eval_metrics.items():
            for metric_name, metric_data in metrics_dict.items():
                da_metric = xr.DataArray(
                    metric_data,
                    coords={'y': ds['y'], 'x': ds['x']},
                    dims=('y', 'x'),
                    name=f'{model_name}_{metric_name}'
                )
                da_metric.attrs['long_name'] = f'{model_name.upper()} {metric_name.upper()} on validation period'
                ds_output_combined[f'{model_name}_{metric_name}'] = da_metric

        # Tambahkan peta orde 'd' ARIMA
        if arima_order_diff_map is not None:
            da_arima_d = xr.DataArray(
                arima_order_diff_map,
                coords={'y': ds['y'], 'x': ds['x']},
                dims=('y', 'x'),
                name='arima_differencing_order_d'
            )
            da_arima_d.attrs['long_name'] = 'ARIMA differencing order (d) determined by auto_arima'
            ds_output_combined['arima_differencing_order_d'] = da_arima_d

        # Tambahkan CRS
        if hasattr(ds, 'rio') and ds.rio.crs:
            ds_output_combined.rio.write_crs(ds.rio.crs, inplace=True)
            ds_output_combined.rio.write_coordinate_system(inplace=True)


        # Simpan NetCDF gabungan
        save_netcdf_output(ds_output_combined, OUTDIRS["netcdf"], "all_models_forecast_and_eval", agg_label, time_labels_forecast)


        # Plot time series agregat
        logging.info("Plotting aggregated time series...")
        plot_ts_aggregate(ds, all_model_predictions, CONFIG["TARGET_VARIABLE"], forecast_steps, validation_gap, OUTDIRS["png"], n_time, agg_label, time_index_all)

        logging.info("===== PIPELINE FINISHED SUCCESSFULLY =====")

    except FileNotFoundError as fnf_err:
        logging.critical(f"CRITICAL ERROR - File Not Found: {fnf_err}")
    except ValueError as val_err:
        logging.critical(f"CRITICAL ERROR - Value Error: {val_err}")
    except ImportError as imp_err:
        logging.critical(f"CRITICAL ERROR - Import Error: {imp_err}. Please ensure all dependencies are installed.")
    except Exception as e:
        logging.critical(f"CRITICAL ERROR - An unexpected error occurred: {e}", exc_info=True)
    finally:
        if 'ds' in locals() and ds is not None:
            ds.close()
            logging.info("Input NetCDF dataset closed.")
        if 'ds_output_combined' in locals() and ds_output_combined is not None:
            ds_output_combined.close()
            logging.info("Output NetCDF dataset closed.")
        logging.info("===== END OF SCRIPT =====")


if __name__ == "__main__":
    main()

2025-05-17 14:27:41,976 INFO:===== PIPELINE STARTED =====
2025-05-17 14:27:41,978 INFO:Changed working directory to: D:\DataPenelitian\Longsor
2025-05-17 14:27:41,979 INFO:Loading NetCDF data from: nc_20250426_1M.nc
2025-05-17 14:27:41,996 INFO:Fallback CRS applied: EPSG:32749
2025-05-17 14:27:41,998 INFO:Starting preprocessing and feature engineering...
2025-05-17 14:27:42,060 INFO:Loaded 1743 landslide points from TitikLongsor_Magelang_2025.shp.

Rasterizing Landslides: 100%|███████████████████████████████████████████████████████| 100/100 [00:00<00:00, 414.13it/s]
2025-05-17 14:27:42,314 INFO:Landslide data processing complete. Feature: LANDSLIDE_COUNT
2025-05-17 14:27:42,333 INFO:Calculated spatial feature: COUNT_SPATIAL_AVG_LAG1
2025-05-17 14:27:42,337 INFO:Added time features: month, sin_month, cos_month.
2025-05-17 14:27:42,338 INFO:Features for ML models: ['COUNT', 'LANDSLIDE_COUNT', 'COUNT_SPATIAL_AVG_LAG1', 'sin_month', 'cos_month']
2025-05-17 14:27:42,338 INFO:Window params s

  0%|          | 0/15 [00:00<?, ?it/s]

Forecast grid ann [1m]:  56%|█████████████████████████████                       | 1080/1935 [1:12:32<09:00,  1.58it/s]2025-05-17 14:27:44,368 INFO:Input NetCDF dataset closed.
2025-05-17 14:27:44,369 INFO:===== END OF SCRIPT =====


[W 2025-05-17 14:27:44,360] Trial 0 failed with parameters: {'n_layers': 2, 'units_layer1': 48, 'units_layer2': 112, 'ann_activation': 'tanh', 'learning_rate': 0.0014415291861597397, 'epochs': 40, 'batch_size': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\ProgramData\miniforge3\envs\proyek_skripsi_lokal\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Wilicious\AppData\Local\Temp\ipykernel_16880\2049637189.py", line 88, in <lambda>
    objective_func = lambda trial: optuna_objective(trial, ds_pixel_features.values, target_col_idx, n_lag, forecast_steps, model_type, validation_gap_tune)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Wilicious\AppData\Local\Temp\ipykernel_16880\2049637189.py", line 58,

KeyboardInterrupt: 